# Pareto Front Analysis for Soil Gas Flux Parameter Optimization

This notebook demonstrates how to use Pareto front analysis to find the optimal balance between measurement uncertainty and model fit quality in soil gas flux calculations.

## Concept

When analyzing chamber CO2 measurements with the H-M model, we face a trade-off:
- **Lower uncertainty range** (more precise estimates) vs. **Better model fit** (higher log-likelihood)
- Different combinations of deadband/cutoff parameters provide different points on this trade-off curve
- The Pareto front identifies the "efficient frontier" where you cannot improve one objective without worsening the other

## Implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from soilgasflux_fcs import HM_model, json_reader
import xarray as xr

def find_pareto_front(x, y, maximize_x=False, maximize_y=False):
    """
    Find the Pareto front for two objectives
    
    Parameters:
    -----------
    x, y : array-like
        Values of the two objectives (uncertainty range vs loglikelihood)
    maximize_x, maximize_y : bool
        Whether to maximize (True) or minimize (False) each objective
    
    Returns:
    --------
    pareto_indices : ndarray
        Indices of points on the Pareto front
    """
    # Copy arrays to avoid modifying originals
    x_values = np.copy(x)
    y_values = np.copy(y)
    
    # Convert maximization to minimization
    if maximize_x:
        x_values = -x_values
    if maximize_y:
        y_values = -y_values
    
    points = np.column_stack((x_values, y_values))
    pareto_indices = []
    
    for i, point in enumerate(points):
        if np.isnan(point).any():
            continue
            
        dominated = False
        for j, other_point in enumerate(points):
            if i != j and not np.isnan(other_point).any():
                # Check if other_point dominates point (smaller is better)
                if (all(other_point <= point) and any(other_point < point)):
                    dominated = True
                    break
        
        if not dominated:
            pareto_indices.append(i)
    
    return np.array(pareto_indices)

## Example Analysis with Synthetic Data

Let's demonstrate the concept with a synthetic dataset where we know the "true" optimal parameters.

In [ ]:
# Generate synthetic MCMC results for demonstration
# In practice, these would come from actual MCMC runs with different deadband/cutoff combinations

np.random.seed(42)
n_parameter_combinations = 50

# Simulate uncertainty range and log-likelihood for different parameter combinations
# Create realistic trade-off: as uncertainty decreases, log-likelihood tends to get worse
base_uncertainty = np.random.uniform(0.1, 2.0, n_parameter_combinations)
base_loglik = np.random.uniform(-30, -5, n_parameter_combinations)

# Add correlation: lower uncertainty often means worse model fit
uncertainty_range = base_uncertainty + 0.3 * np.random.normal(0, 1, n_parameter_combinations)
loglikelihood = base_loglik - 5 * (1 / base_uncertainty) + 2 * np.random.normal(0, 1, n_parameter_combinations)

# Normalize for visualization
norm_uncertainty = (uncertainty_range - np.min(uncertainty_range)) / (np.max(uncertainty_range) - np.min(uncertainty_range))
norm_loglik = (loglikelihood - np.min(loglikelihood)) / (np.max(loglikelihood) - np.min(loglikelihood))

print(f"Generated {n_parameter_combinations} parameter combinations")
print(f"Uncertainty range: {uncertainty_range.min():.3f} - {uncertainty_range.max():.3f}")
print(f"Log-likelihood range: {loglikelihood.min():.1f} - {loglikelihood.max():.1f}")

## Find Pareto Front and Optimal Solution

In [ ]:
# Find Pareto front
# We want to minimize both uncertainty (x) and negative log-likelihood (y)
pareto_indices = find_pareto_front(norm_uncertainty, norm_loglik, maximize_x=False, maximize_y=False)

print(f"Found {len(pareto_indices)} points on Pareto front")

# Find best compromise solution (closest to origin in normalized space)
distances = np.sqrt(norm_uncertainty[pareto_indices]**2 + norm_loglik[pareto_indices]**2)
best_pareto_idx = pareto_indices[np.argmin(distances)]

print(f"\nBest compromise solution:")
print(f"  Index: {best_pareto_idx}")
print(f"  Uncertainty range: {uncertainty_range[best_pareto_idx]:.3f}")
print(f"  Log-likelihood: {loglikelihood[best_pareto_idx]:.1f}")
print(f"  Distance from origin: {distances[np.argmin(distances)]:.3f}")

## Visualization

In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Original scale
ax1.scatter(uncertainty_range, loglikelihood, s=30, c='gray', alpha=0.6, label='All combinations')
ax1.scatter(uncertainty_range[pareto_indices], loglikelihood[pareto_indices], 
           s=50, c='red', alpha=0.8, label='Pareto front', zorder=5)
ax1.scatter(uncertainty_range[best_pareto_idx], loglikelihood[best_pareto_idx], 
           s=100, c='blue', marker='*', label='Best compromise', zorder=10)

ax1.set_xlabel('Uncertainty Range')
ax1.set_ylabel('Log-likelihood')
ax1.set_title('Pareto Front Analysis (Original Scale)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Normalized scale with distance visualization
ax2.scatter(norm_uncertainty, norm_loglik, s=30, c='gray', alpha=0.6, label='All combinations')
ax2.scatter(norm_uncertainty[pareto_indices], norm_loglik[pareto_indices], 
           s=50, c='red', alpha=0.8, label='Pareto front', zorder=5)
ax2.scatter(norm_uncertainty[best_pareto_idx], norm_loglik[best_pareto_idx], 
           s=100, c='blue', marker='*', label='Best compromise', zorder=10)

# Draw line from origin to best solution
ax2.plot([0, norm_uncertainty[best_pareto_idx]], [0, norm_loglik[best_pareto_idx]], 
         'b--', alpha=0.7, label='Distance to origin')
ax2.scatter([0], [0], s=50, c='black', marker='x', label='Origin (ideal)')

ax2.set_xlabel('Normalized Uncertainty Range')
ax2.set_ylabel('Normalized Log-likelihood')
ax2.set_title('Pareto Front Analysis (Normalized Scale)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 1.1)
ax2.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

## Practical Application

In real applications, you would:

1. **Run MCMC analysis** for multiple deadband/cutoff combinations
2. **Calculate metrics** for each combination:
   - Uncertainty range: `np.quantile(dcdt_samples, 0.84) - np.quantile(dcdt_samples, 0.16)`
   - Log-likelihood: `logprob_samples.mean()`
3. **Apply Pareto analysis** to find optimal parameters
4. **Select best solution** based on your priorities (precision vs. model fit)

### Example with Real Data Structure

In [ ]:
# Example workflow with netCDF output from multiprocessing
# This shows how you would apply Pareto analysis to real MCMC results

def analyze_mcmc_results(ds, time_idx=0):
    """
    Apply Pareto front analysis to MCMC results from netCDF dataset
    
    Parameters:
    -----------
    ds : xarray.Dataset
        Dataset with MCMC results including 'dcdt(HM)' and 'logprob(HM)'
    time_idx : int
        Time index to analyze
    
    Returns:
    --------
    dict with optimal parameters and metrics
    """
    
    # Extract data for specific time
    time_data = ds.isel(time=time_idx)
    
    # Calculate uncertainty range (68% confidence interval)
    dcdt_q84 = time_data['dcdt(HM)'].quantile(0.84, dim='MC')
    dcdt_q16 = time_data['dcdt(HM)'].quantile(0.16, dim='MC')
    uncertainty_range = dcdt_q84 - dcdt_q16
    
    # Get log-likelihood (average over MC samples)
    loglikelihood = -time_data['logprob(HM)'].mean(dim='MC')
    
    # Flatten for Pareto analysis
    flat_uncertainty = uncertainty_range.values.flatten()
    flat_loglik = loglikelihood.values.flatten()
    
    # Remove NaN values
    valid_mask = ~(np.isnan(flat_uncertainty) | np.isnan(flat_loglik))
    flat_uncertainty = flat_uncertainty[valid_mask]
    flat_loglik = flat_loglik[valid_mask]
    
    # Normalize
    norm_uncertainty = (flat_uncertainty - np.min(flat_uncertainty)) / (np.max(flat_uncertainty) - np.min(flat_uncertainty))
    norm_loglik = (flat_loglik - np.min(flat_loglik)) / (np.max(flat_loglik) - np.min(flat_loglik))
    
    # Find Pareto front
    pareto_indices = find_pareto_front(norm_uncertainty, norm_loglik, maximize_x=False, maximize_y=False)
    
    # Find best compromise
    distances = np.sqrt(norm_uncertainty[pareto_indices]**2 + norm_loglik[pareto_indices]**2)
    best_pareto_local_idx = np.argmin(distances)
    best_global_idx = np.where(valid_mask)[0][pareto_indices[best_pareto_local_idx]]
    
    # Convert back to cutoff/deadband coordinates
    coords = np.unravel_index(best_global_idx, uncertainty_range.shape)
    optimal_cutoff = time_data.coords['cutoff'][coords[0]].values
    optimal_deadband = time_data.coords['deadband'][coords[1]].values
    
    return {
        'optimal_cutoff': optimal_cutoff,
        'optimal_deadband': optimal_deadband,
        'optimal_uncertainty': flat_uncertainty[pareto_indices[best_pareto_local_idx]],
        'optimal_loglik': flat_loglik[pareto_indices[best_pareto_local_idx]],
        'pareto_points': len(pareto_indices),
        'total_combinations': len(flat_uncertainty)
    }

# Example usage (commented out since we don't have real data loaded)
# ds = xr.open_dataset('path/to/mcmc_results.nc')
# optimal_params = analyze_mcmc_results(ds, time_idx=0)
# print(f"Optimal parameters: cutoff={optimal_params['optimal_cutoff']}, deadband={optimal_params['optimal_deadband']}")

## Key Benefits

1. **Objective Parameter Selection**: Instead of arbitrarily choosing deadband/cutoff, find the mathematically optimal balance

2. **Quality Control**: Identify measurements where no good trade-off exists (all solutions are dominated)

3. **Measurement Optimization**: Understand how different measurement windows affect the precision-accuracy trade-off

4. **Reproducible Analysis**: Consistent, algorithmic approach to parameter selection across studies

## Implementation Notes

- The analysis assumes you want to **minimize both** uncertainty and negative log-likelihood
- Normalization is important when objectives have different scales
- The "best" point can be selected using different criteria (closest to origin, weighted distance, etc.)
- Consider domain expertise when interpreting results - sometimes a slightly dominated solution might be preferable for practical reasons